In [17]:
import numpy as np
import pandas as pd
from RiskLib import calc_returns, value_at_risk, ew_cov, simulation, copula

In [18]:
price = pd.read_csv("data/price.csv")
price.head()

,Date,SPY,AAPL,MSFT,AMZN,TSLA,GOOGL,GOOG,META,NVDA,...,PNC,MDLZ,MO,ADI,GILD,LMT,SYK,GM,TFC,TJX
0,2/14/2022 0:00,432.011322,167.863144,292.261475,155.167007,291.920013,135.526001,135.300003,217.699997,242.443298,...,197.263107,64.592575,46.290192,151.118790,58.443172,377.068665,247.324020,48.182598,60.352272,66.789505
1,2/15/2022 0:00,438.978333,171.749573,297.680664,156.510498,307.476654,136.608505,136.425507,221.000000,264.702484,...,199.789520,64.328896,46.502743,159.029022,58.653572,372.440185,255.490829,49.446892,62.076630,67.673614
2,2/16/2022 0:00,439.470337,171.511032,297.333191,158.100494,307.796661,137.738007,137.487503,216.539993,264.862305,...,201.139511,64.172638,46.770744,165.211960,59.198696,377.000458,256.349976,50.203476,61.990410,65.915215
3,2/17/2022 0:00,430.082642,167.863144,288.626679,154.652496,292.116669,132.539002,132.308502,207.710007,244.841064,...,194.109894,64.514450,47.473091,157.448944,58.615318,378.822571,248.439911,48.600708,59.921181,64.019295
4,2/18/2022 0:00,427.297852,166.292648,285.846893,152.601502,285.660004,130.403000,130.467499,206.160004,236.199127,...,193.984528,64.455841,47.815025,157.380234,58.385788,376.571686,245.042847,48.381695,59.700851,64.981995


In [19]:
arithmetic_returns = calc_returns.return_calculate(price, "DISCRETE", "Date")
arithmetic_returns = arithmetic_returns.drop(arithmetic_returns.columns[0], axis=1)

In [20]:
arithmetic_returns['META'].head()

0    0.015158
1   -0.020181
2   -0.040778
3   -0.007462
4   -0.019790
Name: META, dtype: float64

In [21]:
# Using a normal distribution
var_normal = value_at_risk.var_normal(arithmetic_returns['META'])
es_normal = value_at_risk.ES_normal(arithmetic_returns['META'])
print(var_normal)
print(es_normal)

(0.06561787073963313, 0.06560156967533286)
(0.08228331604039456, 0.08226701497609429)


In [22]:
# Using a normal distribution with an Exponentially Weighted variance
var_normal_EW = value_at_risk.var_normal_ew(arithmetic_returns['META'])
print(var_normal_EW)

(0.06561787073963313, -0.0016069485689090691)


In [23]:
# Using a MLE fitted T distribution
var_mle_t = value_at_risk.var_t_dist(arithmetic_returns['META'])
es_mle_t = value_at_risk.ES_t(arithmetic_returns['META'])
print(var_mle_t)
print(es_mle_t)

(0.05759600524783451, 0.05757970418353424)
(0.08677139118946736, 0.08675509012516709)


In [24]:
# Using a fitted AR(1) model
var_ar1 = value_at_risk.var_ar(arithmetic_returns['META'])
print(var_ar1)

(array([0.06600947]), array([0.06599317]))


In [25]:
# Using a Historic Simulation
var_historic = value_at_risk.var_historic(arithmetic_returns['META'])
print(var_historic)

(0.05463638014667897, 0.0546200790823787)


In [26]:
portfolio = pd.read_csv("data/portfolio.csv")
portfolio.head()

,Portfolio,Stock,Holding
0,A,AAPL,58
1,A,TSLA,10
2,A,JPM,64
3,A,HD,27
4,A,BAC,209


In [27]:
stocks_in_portfolio_A = portfolio['Stock'][portfolio['Portfolio'] == 'A'].values
stocks_in_portfolio_B = portfolio['Stock'][portfolio['Portfolio'] == 'B'].values
stocks_in_portfolio_C = portfolio['Stock'][portfolio['Portfolio'] == 'C'].values

portfolio_A_values = portfolio['Holding'][portfolio['Portfolio'] == 'A'].values * price[stocks_in_portfolio_A].iloc[-1]
portfolio_B_values = portfolio['Holding'][portfolio['Portfolio'] == 'B'].values * price[stocks_in_portfolio_B].iloc[-1]
portfolio_C_values = portfolio['Holding'][portfolio['Portfolio'] == 'C'].values * price[stocks_in_portfolio_C].iloc[-1]

portfolio_total_values = portfolio['Holding'].values * price[portfolio['Stock']].iloc[-1]

portfolio_A_weights = portfolio_A_values / portfolio_A_values.sum()
portfolio_B_weights = portfolio_B_values / portfolio_B_values.sum()
portfolio_C_weights = portfolio_C_values / portfolio_C_values.sum()
portfolio_total_weights = portfolio_total_values / portfolio_total_values.sum()

In [28]:
portfolio_A_returns = arithmetic_returns[stocks_in_portfolio_A]
portfolio_B_returns = arithmetic_returns[stocks_in_portfolio_B]
portfolio_C_returns = arithmetic_returns[stocks_in_portfolio_C]
portfolio_total_returns = arithmetic_returns[portfolio['Stock']]

portfolio_A_ew_cov = ew_cov.ewCovar(portfolio_A_returns, 0.94)
portfolio_B_ew_cov = ew_cov.ewCovar(portfolio_B_returns, 0.94)
portfolio_C_ew_cov = ew_cov.ewCovar(portfolio_C_returns, 0.94)
portfolio_total_ew_cov = ew_cov.ewCovar(portfolio_total_returns, 0.94)

simulated_A_returns = simulation.pca_simulation(portfolio_A_ew_cov)
simulated_B_returns = simulation.pca_simulation(portfolio_B_ew_cov)
simulated_C_returns = simulation.pca_simulation(portfolio_C_ew_cov)
simulated_total_returns = simulation.pca_simulation(portfolio_total_ew_cov)

simulated_portfolioA_return = np.dot(simulated_A_returns, portfolio_A_weights)
simulated_portfolioB_return = np.dot(simulated_B_returns, portfolio_B_weights)
simulated_portfolioC_return = np.dot(simulated_C_returns, portfolio_C_weights)
simulated_portfolio_total_return = np.dot(simulated_total_returns, portfolio_total_weights)

simulated_portfolioA_value_changes = simulated_portfolioA_return * portfolio_A_values.sum()
simulated_portfolioB_value_changes = simulated_portfolioB_return * portfolio_B_values.sum()
simulated_portfolioC_value_changes = simulated_portfolioC_return * portfolio_C_values.sum()
simulated_portfolio_total_value_changes = simulated_portfolio_total_return * portfolio_total_values.sum()

confidence_level = 0.95

portfolioA_var = -np.percentile(simulated_portfolioA_value_changes, (1 - confidence_level) * 100)
print(f"The {confidence_level * 100}% VaR of the portfolio A is ${portfolioA_var:.2f}")
portfolioB_var = -np.percentile(simulated_portfolioB_value_changes, (1 - confidence_level) * 100)
print(f"The {confidence_level * 100}% VaR of the portfolio B is ${portfolioB_var:.2f}")
portfolioC_var = -np.percentile(simulated_portfolioC_value_changes, (1 - confidence_level) * 100)
print(f"The {confidence_level * 100}% VaR of the portfolio C is ${portfolioC_var:.2f}")
portfolio_total_var = -np.percentile(simulated_portfolio_total_value_changes, (1 - confidence_level) * 100)
print(f"The {confidence_level * 100}% VaR of the total portfolio is ${portfolio_total_var:.2f}")

The 95.0% VaR of the portfolio A is $5655.24
The 95.0% VaR of the portfolio B is $4501.56
The 95.0% VaR of the portfolio C is $3777.41
The 95.0% VaR of the total portfolio is $13662.06


In [29]:
portfolio_new = portfolio.copy()
for stock in portfolio_new["Stock"]:
    portfolio_new.loc[portfolio_new['Stock'] == stock, 'Starting Price'] = price.iloc[-1][stock]

portfolio_new.loc[portfolio_new['Portfolio'].isin(['A', 'B']), 'Distribution'] = 'T'
portfolio_new.loc[portfolio_new['Portfolio'] == 'C', 'Distribution'] = 'Normal'
print(portfolio_new.head())

  Portfolio Stock  Holding  Starting Price Distribution
0         A  AAPL       58      150.639999            T
1         A  TSLA       10      207.320007            T
2         A   JPM       64      140.419998            T
3         A    HD       27      315.549988            T
4         A   BAC      209       35.720001            T


In [30]:
portfolio_A = portfolio_new[portfolio_new['Portfolio'] == 'A'].copy()
portfolio_A = portfolio_A.drop('Portfolio', axis=1)
portfolio_B = portfolio_new[portfolio_new['Portfolio'] == 'B'].copy()
portfolio_B = portfolio_B.drop('Portfolio', axis=1)
portfolio_C = portfolio_new[portfolio_new['Portfolio'] == 'C'].copy()
portfolio_C = portfolio_C.drop('Portfolio', axis=1)

In [31]:
risk_A = copula.simulateCopula(portfolio_A, portfolio_A_returns)
risk_B = copula.simulateCopula(portfolio_B, portfolio_B_returns)
risk_C = copula.simulateCopula(portfolio_C, portfolio_C_returns)
risk_total = copula.simulateCopula(portfolio_new.drop('Portfolio', axis=1), portfolio_total_returns)
print('Portfolio A:')
print(risk_A[risk_A['Stock']=='Total'])
print('Portfolio B:')
print(risk_B[risk_B['Stock']=='Total'])
print('Portfolio C:')
print(risk_C[risk_C['Stock']=='Total'])
print('Total Portfolio:')
print(risk_total[risk_total['Stock']=='Total'])

Portfolio A:
    Stock        VaR95          ES95
35  Total  7929.631108  10581.948695
Portfolio B:
    Stock        VaR95         ES95
32  Total  6795.790553  8993.930764
Portfolio C:
    Stock        VaR95         ES95
32  Total  5883.509289  7396.565108
Total Portfolio:
    Stock         VaR95          ES95
99  Total  20601.346728  26632.087012
